**OpenTable reservation time scraper**

Link: https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3146&shouldUseLatLongSearch=false&originCorrelationId=6b9e96ee-1ae5-4791-a91a-e60d7091faf0&corrid=77e4b75f-cde3-4d45-b7b1-14790e7f372f&queryUnderstandingType=none&showMap=true&sortBy=web_conversion

31st Jan 2026, 23:30, 2px

In [1]:
# load packages
!pip install selenium webdriver-manager beautifulsoup4

import requests
from bs4 import BeautifulSoup
import time
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from functools import reduce
import re

In [ ]:
# aberdeen
import csv
import time
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3029&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("restaurants.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)


In [16]:
# birmingham
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3146&shouldUseLatLongSearch=false&originCorrelationId=6b9e96ee-1ae5-4791-a91a-e60d7091faf0&corrid=77e4b75f-cde3-4d45-b7b1-14790e7f372f&queryUnderstandingType=none&showMap=true&sortBy=web_conversion"

def scrape_opentable_birmingham_one_click(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicked = False  # flag to click next page only once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if not next_clicked:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicked = True
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # already clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'birmingham_west_midlands.csv'...")
        with open("birmingham_west_midlands.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_birmingham_one_click(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [18]:
# brighton
import csv
import time
from bs4 import BeautifulSoup
import undetected_chromedriver as uc

URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3089&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=550d3ccc-a24c-4a57-bd6e-05d0487accdf"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("brighton.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 50 restaurant cards.

 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing 

In [19]:
# edinburgh
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3191&shouldUseLatLongSearch=false&originCorrelationId=f1b4cbb5-e6d8-4c0b-902b-3bce62b50602&corrid=0f2529b5-29dd-4066-9077-6f2333afc0aa&queryUnderstandingType=none&showMap=true&sortBy=web_conversion"

def scrape_opentable_edinburgh(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 3  # click next page 3 times for 4 pages total

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE UP TO 3 TIMES -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked 3 times, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'edinburgh.csv'...")
        with open("edinburgh.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_edinburgh(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [20]:
# essex
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3125&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=2f122bc1-0b38-45c8-9a50-126f2d596f96"

def scrape_opentable_essex(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'essex.csv'...")
        with open("essex.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_essex(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [21]:
# glasgow
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3188&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=5c236a4f-f291-4199-8d33-6945dbf4ce9e"

def scrape_opentable_glasgow(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'glasgow.csv'...")
        with open("glasgow.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_glasgow(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [22]:
# gloucestershire
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3185&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=0c1698da-32be-4844-82f5-3d9dc818ba40"

def scrape_opentable_gloucestershire(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'gloucestershire.csv'...")
        with open("gloucestershire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_gloucestershire(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [24]:
# hampshire
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3092&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=937a2167-7929-4f1f-9031-8c4da62da22a"

def scrape_opentable_hampshire(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'hampshire.csv'...")
        with open("hampshire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_hampshire(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [25]:
# hertfordshire
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3128&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=5576cd00-921e-4e3f-84e7-3976d5efe639"

def scrape_opentable_hertfordshire(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # only click once

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE ONCE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked once, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'hertfordshire.csv'...")
        with open("hertfordshire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_hertfordshire(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [26]:
# kent
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3098&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=d0d4ad73-ecca-4eb8-9098-847ad558744c"

def scrape_opentable_kent(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 3  # click 3 times for 4 pages total

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button found, finished scraping.")
                    break
            else:
                break  # clicked 3 times, stop here

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'kent.csv'...")
        with open("kent.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_kent(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [27]:
# liverpool
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3176&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=004ce382-43c3-4c64-bccc-4ab5d52c374c"
def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("liverpool.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 50 restaurant cards.

 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing 

In [3]:
# city of london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5289&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=720df919-a753-40d7-b124-18b17e7533d0"

def scrape_opentable_city_of_london(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # keep browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 2  # <── CLICK 2 TIMES

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL PAGE SLOWLY -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract restaurant data
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK "NEXT PAGE" -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)

                except TimeoutException:
                    print("Next page button not found — stopping.")
                    break
            else:
                break

        # ----- SAVE CSV -----
        print("\n[5/6] Saving results to 'city_of_london.csv'...")
        with open("city_of_london.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_city_of_london(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [4]:
# east london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5292&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=e3511a02-7733-4c02-8101-8d9d4b6284fe"

def scrape_opentable_east_london(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 3  # <── CLICK 3 TIMES

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL PAGE SLOWLY -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract restaurant data
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK "NEXT PAGE" BUTTON -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)

                except TimeoutException:
                    print("Next page button not found — stopping.")
                    break
            else:
                break

        # ----- SAVE RESULTS -----
        print("\n[5/6] Saving results to 'east_london.csv'...")
        with open("east_london.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_east_london(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [5]:
# north london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5298&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=7e96b075-24a7-4294-aba9-2bd5ce27bc1e"

def scrape_opentable_north_london(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # show browser

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 4  # CLICK 4 TIMES

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract restaurant data
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")

                time.sleep(0.05)

            # ----- CLICK NEXT PAGE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)

                except TimeoutException:
                    print("Next page button NOT found — stopping early.")
                    break
            else:
                break

        # SAVE CSV
        print("\n[5/6] Saving results to 'north_london.csv'...")
        with open("north_london.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_north_london(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [6]:
# north west london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5301&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=c5d99e02-8dff-487b-8429-c15662c52f46"

def scrape_opentable_north_west_london(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 1  # CLICK **ONCE**

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOW -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract restaurants
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")

                time.sleep(0.05)

            # ----- NEXT BUTTON -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)

                except TimeoutException:
                    print("Next page button NOT found — stopping early.")
                    break
            else:
                break

        # SAVE RESULTS
        print("\n[5/6] Saving results to 'north_west_london.csv'...")
        with open("north_west_london.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_north_west_london(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [7]:
# south east london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5304&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=f2b1fc34-e89c-45c3-91d6-11dbf1ab1f6f"

def scrape_opentable_south_east_london(url):
    print("[1/6] Creating Chrome options…")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # show browser

    print("[2/6] Launching Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 3

    try:
        driver.get(url)

        while True:
            print(f"\n[3/6] Page {page_number} - loading and scrolling")
            time.sleep(3)

            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error with restaurant {idx}: {e}")
                time.sleep(0.05)

            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]'))
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("No next page button — ending.")
                    break
            else:
                break

        print("\n[4/6] Saving results to 'south_east_london.csv'...")
        with open("south_east_london.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[5/6] Done! Total restaurants: {len(all_results)}")
    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_south_east_london(URL)


[1/6] Creating Chrome options…
[2/6] Launching Chrome...

[3/6] Page 1 - loading and scrolling
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → 

In [8]:
# south west london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5316&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=772db3a0-5cf2-4b7a-8847-664e233678c8"

def scrape_opentable_south_west_london(url):
    print("[1/6] Creating fresh Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []

    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 7  # CLICK 7 TIMES

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
            print("[4/6] Scrolling page to load all restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract info
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            # ----- CLICK NEXT PAGE -----
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — stopping early.")
                    break
            else:
                break

        # ----- SAVE RESULTS TO CSV -----
        print("\n[5/6] Saving results to 'south_west_london.csv'...")
        with open("south_west_london.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_south_west_london(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling page to load all restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing 

In [9]:
# west end london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5322&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=8da66c7e-b325-4d9f-9fe7-d39371863a87"

def scrape_opentable_west_end_london(url):
    print("[1/6] Creating fresh Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    try:
        driver.get(url)
        page_number = 1
        next_clicks_remaining = 11  # CLICK 11 TIMES

        while True:
            print(f"\n[3/6] Processing page {page_number}...")
            time.sleep(3)

            # Scroll to load all restaurants
            print("[4/6] Scrolling to load restaurants...")
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Parse page
            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurant cards on page {page_number}.")

            # Extract data
            for idx, card in enumerate(cards, start=1):
                print(f" → Processing restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")

                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })

                except Exception as e:
                    print(f"   ! Error with restaurant {idx}: {e}")

                time.sleep(0.05)

            # Click next page if remaining
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)

                except TimeoutException:
                    print("Next-page button not found — stopping early.")
                    break
            else:
                break

        # Save to CSV
        print("\n[5/6] Saving results to 'west_end_london.csv'...")
        with open("west_end_london.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"[6/6] Scraping complete! Total restaurants: {len(all_results)}\n")

    finally:
        driver.quit()

    return all_results


if __name__ == "__main__":
    scrape_opentable_west_end_london(URL)


[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...

[3/6] Processing page 1...
[4/6] Scrolling to load restaurants...
Found 50 restaurant cards on page 1.
 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing restaurant 26/50
 → Processing restauran

In [11]:
# west london
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5325&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=f9fec981-fda5-451e-a4b3-8e8a890b647f"

def scrape_opentable_west_london(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 4  # CLICK 4 TIMES

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            if next_clicks_remaining > 0:
                try:
                    next_btn = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_btn)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — exiting.")
                    break
            else:
                break

        # Save CSV
        print("\nSaving results to 'west_london.csv'...")
        with open("west_london.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print(f"Done! Total restaurants scraped: {len(all_results)}")
    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_west_london(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [12]:
# docklands and canary wharf
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=72&regionIds%5B%5D=5343&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=4e88e26d-ae6a-45a8-9346-91330d78cf18"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("docklands_canary_wharf.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 50 restaurant cards.

 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing 

In [13]:
# manchester
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3200&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=3566c9b2-2db9-4aaa-b673-cbab1673eadc"

def scrape_opentable_manchester(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 2  # CLICK 2 TIMES

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing card {idx}: {e}")
                time.sleep(0.05)

            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — exiting.")
                    break
            else:
                break

        print("\nSaving to 'manchester.csv' ...")
        with open("manchester.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Done! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_manchester(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [14]:
# newcastle
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3116&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=1cadafb0-b555-4ec0-8f81-3bad74a184ea"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("newcastle.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 50 restaurant cards.

 → Processing restaurant 1/50
 → Processing restaurant 2/50
 → Processing restaurant 3/50
 → Processing restaurant 4/50
 → Processing restaurant 5/50
 → Processing restaurant 6/50
 → Processing restaurant 7/50
 → Processing restaurant 8/50
 → Processing restaurant 9/50
 → Processing restaurant 10/50
 → Processing restaurant 11/50
 → Processing restaurant 12/50
 → Processing restaurant 13/50
 → Processing restaurant 14/50
 → Processing restaurant 15/50
 → Processing restaurant 16/50
 → Processing restaurant 17/50
 → Processing restaurant 18/50
 → Processing restaurant 19/50
 → Processing restaurant 20/50
 → Processing restaurant 21/50
 → Processing restaurant 22/50
 → Processing restaurant 23/50
 → Processing restaurant 24/50
 → Processing restaurant 25/50
 → Processing 

In [15]:
#norfolk + suffolk
URL = ("https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00"
       "&covers=2&metroId=3131&regionIds%5B%5D=&neighborhoodIds%5B%5D="
       "&term=&shouldUseLatLongSearch=false"
       "&originCorrelationId=2c1c6482-c4fa-45f4-8cef-8c13f0a8d694")

def scrape_opentable_norfolk_suffolk(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # keep browser visible

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 1  # click once

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load all restaurant cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find(
                        "a",
                        attrs={"href": lambda x: x and "#reviews" in x}
                    )
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing restaurant {idx}: {e}")
                time.sleep(0.05)

            # Click next page if still allowed
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)

                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — ending.")
                    break
            else:
                break

        print("\nSaving results to 'norfolk_suffolk.csv'...")
        with open("norfolk_suffolk.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Scraping complete! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_norfolk_suffolk(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [16]:
# northern ireland
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3026&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=b2f54ad5-3d58-4346-968a-0f359d983aaf"

def scrape_opentable_uc_scroll(url):
    print("[1/6] Creating fresh Chrome options...")
    
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching undetected Chrome...")
    driver = uc.Chrome(options=options, version_main=142)

    try:
        print("[3/6] Loading OpenTable page...")
        driver.get(url)
        time.sleep(3)

        # ----- SCROLL SLOWLY TO LOAD ALL CARDS -----
        print("[4/6] Scrolling page to load all restaurants...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollBy(0, 500);")  # scroll down 500px
            time.sleep(1)  # wait for lazy-loaded content
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break  # reached bottom
            last_height = new_height

        print("[5/6] Extracting restaurant data...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
        print(f"Found {len(cards)} restaurant cards.\n")

        results = []

        for idx, card in enumerate(cards, start=1):
            print(f" → Processing restaurant {idx}/{len(cards)}")
            try:
                restaurant_id = card.get("data-rid")

                name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                name = name_tag.get_text(strip=True) if name_tag else None

                rating_tag = card.select_one(".tSiVMQB9es0-")
                rating = rating_tag.get_text(strip=True) if rating_tag else None

                review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                reviews = review_tag.get_text(strip=True) if review_tag else None

                price_tag = card.select_one(".Vk-xtpOrXcE-")
                price = price_tag.get_text(strip=True) if price_tag else None

                cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                slots = [
                    slot.get_text(strip=True)
                    for slot in card.select('[data-test^="time-slot"] div')
                    if slot.get_text(strip=True)
                ]

                results.append({
                    "restaurant_id": restaurant_id,
                    "name": name,
                    "rating": rating,
                    "reviews": reviews,
                    "price": price,
                    "cuisine_location": cuisine_location,
                    "timeslots": ", ".join(slots),
                })

            except Exception as e:
                print(f"   ! Error processing card {idx}: {e}")

            time.sleep(0.05)

        print("\n[6/6] Saving results to CSV...")
        with open("northern_ireland.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(results)

        print("Scraping complete! Saved as restaurants.csv\n")

    finally:
        driver.quit()

    return results


if __name__ == "__main__":
    scrape_opentable_uc_scroll(URL)

[1/6] Creating fresh Chrome options...
[2/6] Launching undetected Chrome...
[3/6] Loading OpenTable page...
[4/6] Scrolling page to load all restaurants...
[5/6] Extracting restaurant data...
Found 38 restaurant cards.

 → Processing restaurant 1/38
 → Processing restaurant 2/38
 → Processing restaurant 3/38
 → Processing restaurant 4/38
 → Processing restaurant 5/38
 → Processing restaurant 6/38
 → Processing restaurant 7/38
 → Processing restaurant 8/38
 → Processing restaurant 9/38
 → Processing restaurant 10/38
 → Processing restaurant 11/38
 → Processing restaurant 12/38
 → Processing restaurant 13/38
 → Processing restaurant 14/38
 → Processing restaurant 15/38
 → Processing restaurant 16/38
 → Processing restaurant 17/38
 → Processing restaurant 18/38
 → Processing restaurant 19/38
 → Processing restaurant 20/38
 → Processing restaurant 21/38
 → Processing restaurant 22/38
 → Processing restaurant 23/38
 → Processing restaurant 24/38
 → Processing restaurant 25/38
 → Processing 

In [17]:
# oxford
URL = ("https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2"
       "&metroId=3101&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&"
       "shouldUseLatLongSearch=false&originCorrelationId=2089c44e-bb36-477d-bc48-9bfd37f7f2f5")

def scrape_opentable_oxford(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 1  # click once

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load restaurant cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing restaurant {idx}: {e}")
                time.sleep(0.05)

            # Click next page if remaining
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — ending.")
                    break
            else:
                break

        print("\nSaving results to 'oxford.csv'...")
        with open("oxford.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Scraping complete! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_oxford(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [18]:
# south wales
URL = ("https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3017&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=8c0f415e-fdf7-4d39-954f-ed4051a210a7")

def scrape_opentable_oxford(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 1  # click once

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load restaurant cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing restaurant {idx}: {e}")
                time.sleep(0.05)

            # Click next page if remaining
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — ending.")
                    break
            else:
                break

        print("\nSaving results to 'oxford.csv'...")
        with open("south_wales.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Scraping complete! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_oxford(URL)

[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [19]:
# surrey
URL = ("https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3107"
       "&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false"
       "&originCorrelationId=29ce2166-f8e3-4e67-befa-f3632781467e")

def scrape_opentable_surrey(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 2  # click twice

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load restaurant cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find(
                        "a",
                        attrs={"href": lambda x: x and "#reviews" in x}
                    )
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing restaurant {idx}: {e}")
                time.sleep(0.05)

            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — ending.")
                    break
            else:
                break

        print("\nSaving results to 'surrey.csv'...")
        with open("surrey.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Scraping complete! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_surrey(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [23]:
# west sussex
URL = "https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00&covers=2&metroId=3215&regionIds%5B%5D=&neighborhoodIds%5B%5D=&term=&shouldUseLatLongSearch=false&originCorrelationId=50b89764-4c7b-4436-b7d7-0ecd09f278cc"

def scrape_opentable_west_sussex(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # make browser visible

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 1  # only one click

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load all restaurant cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find("a", attrs={"href": lambda x: x and "#reviews" in x})
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing restaurant {idx}: {e}")
                time.sleep(0.05)

            # If we still have a click left, attempt next page
            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — ending.")
                    break
            else:
                break

        print("\nSaving results to 'west_sussex.csv'...")
        with open("west_sussex.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Scraping complete! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_west_sussex(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [22]:
# yorkshire
URL = ("https://www.opentable.co.uk/s?dateTime=2026-01-31T23%3A30%3A00"
       "&covers=2&metroId=3023&regionIds%5B%5D=&neighborhoodIds%5B%5D="
       "&term=&shouldUseLatLongSearch=false"
       "&originCorrelationId=3318a928-f559-4d62-b032-886547055967")

def scrape_opentable_yorkshire(url):
    print("[1/6] Creating Chrome options...")
    options = uc.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.headless = False  # visible browser

    print("[2/6] Launching browser...")
    driver = uc.Chrome(options=options, version_main=142)

    all_results = []
    page_number = 1
    next_clicks_remaining = 3  # click 3 times

    try:
        driver.get(url)

        while True:
            print(f"\n[Processing page {page_number}]")
            time.sleep(3)

            # Scroll slowly to load all restaurant cards
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(1)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = BeautifulSoup(driver.page_source, "html.parser")
            cards = soup.find_all("div", attrs={"data-test": "restaurant-card"})
            print(f"Found {len(cards)} restaurants on page {page_number}")

            for idx, card in enumerate(cards, start=1):
                print(f" → Restaurant {idx}/{len(cards)}")
                try:
                    restaurant_id = card.get("data-rid")
                    name_tag = card.find("a", attrs={"data-test": "res-card-name"})
                    name = name_tag.get_text(strip=True) if name_tag else None

                    rating_tag = card.select_one(".tSiVMQB9es0-")
                    rating = rating_tag.get_text(strip=True) if rating_tag else None

                    review_tag = card.find(
                        "a",
                        attrs={"href": lambda x: x and "#reviews" in x}
                    )
                    reviews = review_tag.get_text(strip=True) if review_tag else None

                    price_tag = card.select_one(".Vk-xtpOrXcE-")
                    price = price_tag.get_text(strip=True) if price_tag else None

                    cuisine_tag = card.select_one('[data-test="cuisine-and-location"]')
                    cuisine_location = cuisine_tag.get_text(strip=True) if cuisine_tag else None

                    slots = [
                        slot.get_text(strip=True)
                        for slot in card.select('[data-test^="time-slot"] div')
                        if slot.get_text(strip=True)
                    ]

                    all_results.append({
                        "restaurant_id": restaurant_id,
                        "name": name,
                        "rating": rating,
                        "reviews": reviews,
                        "price": price,
                        "cuisine_location": cuisine_location,
                        "timeslots": ", ".join(slots),
                    })
                except Exception as e:
                    print(f"   ! Error processing restaurant {idx}: {e}")
                time.sleep(0.05)

            if next_clicks_remaining > 0:
                try:
                    next_button = WebDriverWait(driver, 6).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'a[aria-label="Go to the next page"]')
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", next_button)
                    next_clicks_remaining -= 1
                    page_number += 1
                    time.sleep(3)
                except TimeoutException:
                    print("Next page button not found — ending.")
                    break
            else:
                break

        print("\nSaving results to 'yorkshire.csv'...")
        with open("yorkshire.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=[
                "restaurant_id", "name", "rating", "reviews",
                "price", "cuisine_location", "timeslots"
            ])
            writer.writeheader()
            writer.writerows(all_results)

        print("Scraping complete! Total restaurants:", len(all_results))

    finally:
        driver.quit()


if __name__ == "__main__":
    scrape_opentable_yorkshire(URL)


[1/6] Creating Chrome options...
[2/6] Launching browser...

[Processing page 1]
Found 50 restaurants on page 1
 → Restaurant 1/50
 → Restaurant 2/50
 → Restaurant 3/50
 → Restaurant 4/50
 → Restaurant 5/50
 → Restaurant 6/50
 → Restaurant 7/50
 → Restaurant 8/50
 → Restaurant 9/50
 → Restaurant 10/50
 → Restaurant 11/50
 → Restaurant 12/50
 → Restaurant 13/50
 → Restaurant 14/50
 → Restaurant 15/50
 → Restaurant 16/50
 → Restaurant 17/50
 → Restaurant 18/50
 → Restaurant 19/50
 → Restaurant 20/50
 → Restaurant 21/50
 → Restaurant 22/50
 → Restaurant 23/50
 → Restaurant 24/50
 → Restaurant 25/50
 → Restaurant 26/50
 → Restaurant 27/50
 → Restaurant 28/50
 → Restaurant 29/50
 → Restaurant 30/50
 → Restaurant 31/50
 → Restaurant 32/50
 → Restaurant 33/50
 → Restaurant 34/50
 → Restaurant 35/50
 → Restaurant 36/50
 → Restaurant 37/50
 → Restaurant 38/50
 → Restaurant 39/50
 → Restaurant 40/50
 → Restaurant 41/50
 → Restaurant 42/50
 → Restaurant 43/50
 → Restaurant 44/50
 → Restaurant 45/

In [31]:
# load in the files
aberdeen = pd.read_csv("restaurants.csv")
birmingham = pd.read_csv("birmingham_west_midlands.csv")
brighton = pd.read_csv("brighton.csv")
edinburgh = pd.read_csv("edinburgh.csv")
essex = pd.read_csv("essex.csv")
glasgow = pd.read_csv("glasgow.csv")
gloucestershire = pd.read_csv("gloucestershire.csv")
hampshire = pd.read_csv("hampshire.csv")
hertfordshire = pd.read_csv("hertfordshire.csv")
kent = pd.read_csv("kent.csv")
liverpool = pd.read_csv("liverpool.csv")
city_of_london = pd.read_csv("city_of_london.csv")
east_london = pd.read_csv("east_london.csv")
north_london = pd.read_csv("north_london.csv")
north_west_london = pd.read_csv("north_west_london.csv")
south_east_london = pd.read_csv("south_east_london.csv")
south_west_london = pd.read_csv("south_west_london.csv")
west_end_london = pd.read_csv("west_end_london.csv")
west_london = pd.read_csv("west_london.csv")
docklands_canary_wharf = pd.read_csv("docklands_canary_wharf.csv")
manchester = pd.read_csv("manchester.csv")
newcastle = pd.read_csv("newcastle.csv")
northern_ireland = pd.read_csv("northern_ireland.csv")
oxford = pd.read_csv("oxford.csv")
south_wales = pd.read_csv("south_wales.csv")
surrey = pd.read_csv("surrey.csv")
west_sussex = pd.read_csv("west_sussex.csv")
yorkshire = pd.read_csv("yorkshire.csv")

In [51]:
# label and concatanate
datasets = [
    ('aberdeen', aberdeen),
    ('birmingham', birmingham),
    ('brighton', brighton),
    ('edinburgh', edinburgh),
    ('essex', essex),
    ('glasgow', glasgow),
    ('gloucestershire', gloucestershire),
    ('hampshire', hampshire),
    ('hertfordshire', hertfordshire),
    ('kent', kent),
    ('liverpool', liverpool),
    ('city_of_london', city_of_london),
    ('east_london', east_london),
    ('north_london', north_london),
    ('north_west_london', north_west_london),
    ('south_east_london', south_east_london),
    ('south_west_london', south_west_london),
    ('west_end_london', west_end_london),
    ('west_london', west_london),
    ('docklands_canary_wharf', docklands_canary_wharf),
    ('manchester', manchester),
    ('newcastle', newcastle),
    ('northern_ireland', northern_ireland),
    ('oxford', oxford),
    ('south_wales', south_wales),
    ('surrey', surrey),
    ('west_sussex', west_sussex),
    ('yorkshire', yorkshire)
]

for name, df in datasets:
    df['location'] = name

uk_restaurants = pd.concat([df for _, df in datasets], ignore_index=True)

In [55]:
# cleaning
uk_restaurants = uk_restaurants.drop(columns=['rating', 'reviews'])

uk_restaurants = uk_restaurants.dropna(subset=['timeslots'])
uk_restaurants = uk_restaurants[uk_restaurants['timeslots'].str.strip() != '']

allowed_times = ['22:30', '23:00', '23:30', '00:00']

def has_allowed_time(timeslot_str):
    for t in allowed_times:
        if t in timeslot_str:
            return True
    return False

uk_restaurants = uk_restaurants[uk_restaurants['timeslots'].apply(has_allowed_time)]

uk_restaurants = uk_restaurants.reset_index(drop=True)

uk_restaurants['price'] = uk_restaurants['price'].str.replace('Price:\s*', '', regex=True)

valid_times = ['22:30', '23:00', '23:30', '00:00', '00:30', '01:00', '01:30', '02:00', '02:30']

def filter_timeslots(timeslots):
    if pd.isna(timeslots):
        return None
    times = [t.strip() for t in timeslots.split(',')]
    times = [t for t in times if any(t.startswith(v) for v in valid_times)]
    if times:
        return ', '.join(times)
    else:
        return None

uk_restaurants['timeslots'] = uk_restaurants['timeslots'].apply(filter_timeslots)

uk_restaurants['timeslots'] = uk_restaurants['timeslots'].str.replace('*', '', regex=False)

uk_restaurants = uk_restaurants.dropna(subset=['timeslots'])

def split_cuisine_location(text):
    if pd.isna(text):
        return pd.Series([None, None])
    parts = [part.strip() for part in text.split('•') if part.strip()]
    cuisine = parts[0] if len(parts) > 0 else None
    location = parts[1] if len(parts) > 1 else None
    return pd.Series([cuisine, location])

uk_restaurants[['cuisine', 'city']] = uk_restaurants['cuisine_location'].apply(split_cuisine_location)

uk_restaurants = uk_restaurants.drop(columns=['cuisine_location'])

uk_restaurants = uk_restaurants.reset_index(drop=True)

In [77]:
uk_restaurants

,restaurant_id,name,price,timeslots,location,cuisine,city
0,216807,The Spiritualist Aberdeen,Moderate,"23:00, 23:30, 00:00",aberdeen,Scottish,Aberdeen
1,425358,Wild Goosenew,Expensive,"23:00, 23:30, 00:00",aberdeen,British,Aberdeen
2,292485,Faffless,Moderate,23:00,aberdeen,Wine Bar,Aberdeen
3,362364,The Tilted Wignew,Moderate,23:00,aberdeen,Pub,Aberdeen
4,419469,Grosvenor Casino Aberdeennew,Expensive,"23:00, 23:30, 00:00",aberdeen,British,Aberdeen
...,...,...,...,...,...,...,...
890,444906,Tavern On The Quaysnew,Moderate,"23:00, 23:30",yorkshire,British,Sheffield
891,280455,El Toro Bawtry,Moderate,22:30,yorkshire,Spanish,Bawtry
892,299172,Raj Pavilion,Moderate,22:30,yorkshire,Indian,Hull
893,419370,Grosvenor Casino Sheffieldnew,Expensive,"23:00, 23:30, 00:00",yorkshire,British,Sheffield


In [78]:
# save to csv
uk_restaurants.to_csv("uk_restaurants.csv", index=False, encoding='utf-8')

In [79]:
# filtering further to get 11pm and midnight kitchens
uk_restaurants['timeslot_list'] = uk_restaurants['timeslots'].str.split(',\s*')

_23_00 = uk_restaurants[uk_restaurants['timeslot_list'].apply(lambda x: '23:00' in x)].copy()

_24_00 = uk_restaurants[uk_restaurants['timeslot_list'].apply(lambda x: '00:00' in x)].copy()

_23_00.drop(columns=['timeslot_list'], inplace=True)
_24_00.drop(columns=['timeslot_list'], inplace=True)

print(f"Rows with 23:00: {_23_00.shape[0]}")
print(f"Rows with 00:00: {_24_00.shape[0]}")

Rows with 23:00: 441
Rows with 00:00: 109


In [82]:
# saving to csv
_23_00.to_csv("_23_00.csv", index=False, encoding='utf-8')